In [104]:
import riskfolio as rf

import matplotlib.pyplot as plt
import numpy as np
import pickle
import random

import pandas as pd

import gym
from gym import spaces

from stable_baselines3 import PPO, A2C, TD3, SAC, DDPG, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecFrameStack, SubprocVecEnv
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

import torch

In [105]:
def softmax_normalization(actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output

## Definition of gym Environment

In [123]:
class PortfolioAllocationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, data_file,
                 sector = None,
                 objective = 'max_return',
                 risk_measure = 'MV',
                 drop_close = True,
                 observations_range = (0, 1259)):
        """
        Инициализация среды
        """
        super(PortfolioAllocationEnv, self).__init__()

        with open(data_file, 'rb') as f:
            self.data = pickle.load(f)
            if sector is not None:
                self.data = list(filter(lambda stock: stock['sector'] == sector, self.data))
            if drop_close:
                for stock in self.data:
                    stock['data'] = stock['data'].drop(['Close'], axis=1)

        self.stock_dim = self.data[0]['data'].shape[1]

        close_price_changes = {stock['ticker']: stock['data']['Close_Change'] for stock in self.data}
        self.close_prices_pct = pd.DataFrame(close_price_changes)

        self.tickers = [stock['ticker'] for stock in self.data]
        self.n_assets = len(self.data)

        self.objective = objective
        self.risk_measure = risk_measure

        self.rewards_memory = None
        self.actions_memory = None

        self.portfolio_value = 0
        self.observations_range = observations_range
        self.current_step = observations_range[0]

        self.weights = np.ones(self.n_assets) / self.n_assets

        self.reward_range = (-10, 10)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(self.n_assets,))
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets, self.stock_dim + 1))

    def step(self, action):
        """
        Выполнение одного шага
        """
        # assert self.action_space.contains(action), "Invalid action"

        # Получаем текущее состояние
        state = self._get_state()

        # Вычисляем награду
        reward = self._get_reward(action)

        # Обновляем текущий шаг
        self.current_step += 1

        # Проверяем, достигнут ли конец эпизода
        done = self.current_step >= self.observations_range[1]

        if done:
            print(self.render())

        # Возвращаем состояние, награду и флаг окончания эпизода
        return state, reward, done, {}

    def reset(self):
        """
        Сброс среды в начальное состояние
        """
        self.portfolio_value = 0
        self.rewards_memory = []
        self.current_step = self.observations_range[0]
        self.actions_memory = []
        self.weights = np.ones(self.n_assets) / self.n_assets
        return self._get_state()

    def render(self, mode='human'):
        """
        Вывод информации о среде
        """
        return {
            'min_reward': min(self.rewards_memory),
            'max_reward': max(self.rewards_memory),
            'mean_reward': np.array(self.rewards_memory).mean(),
            'portfolio_value': self.portfolio_value,
            'weights_q0.25': np.quantile(self.weights, 0.25),
            'weights_q0.5': np.quantile(self.weights, 0.5),
            'weights_q0.75': np.quantile(self.weights, 0.75),
            'weights_max': np.max(self.weights),
            'weights_std': self.weights.std()
        }

    def _get_state(self):
        """
        Получение текущего состояния
        """
        values = [stock['data'].iloc[self.current_step].values for stock in self.data]
        observation = np.vstack(values)
        observation = np.concatenate([observation, self.weights[:, None]], axis=1).astype(np.float32)
        observation = np.nan_to_num(observation)
        return observation

    def _get_reward(self, action):
        """
        Вычисление награды
        """

        self.weights += action
        self.weights = softmax_normalization(self.weights)

        self.actions_memory.append(action)

        prices_change_hist = self.close_prices_pct.iloc[self.observations_range[0]:self.current_step + 1]
        current_prices = self.close_prices_pct.iloc[self.current_step].values

        portfolio_value_change = current_prices @ self.weights
        self.portfolio_value += portfolio_value_change

        reward = 0

        if self.objective == 'sharpe':
            mu = rf.mean_vector(prices_change_hist, method='hist')
            cov = rf.covar_matrix(prices_change_hist, method='shrunk')
            reward = rf.Sharpe(self.weights, mu, cov, prices_change_hist, rm=self.risk_measure)
        elif self.objective == 'price_change':
            # reward = portfolio_value_change
            reward = portfolio_value_change
        elif self.objective == 'min_risk':
            mu = rf.mean_vector(prices_change_hist, method='hist')
            cov = rf.covar_matrix(prices_change_hist, method='shrunk')
            reward = -rf.Sharpe_Risk(self.weights, cov, mu, rm=self.risk_measure)

        if np.isinf(reward):
            reward = 0

        self.rewards_memory.append(reward)
        # print('act', self.current_step, np.isnan(action).sum(), np.array(self.rewards_memory).mean())

        return reward

In [124]:
train_env = PortfolioAllocationEnv(data_file='data/sp500_components_7y.pickle',
                                   sector='Information Technology',
                                   objective='price_change',
                                   drop_close=False,
                                   observations_range=(0, 1260))  # 252 x 5 = 1260, 252 x 3 = 756
check_env(train_env, skip_render_check=False)

/home/hivaze/Documents/CODE-W/PyCharm Projects/portfolio_optimization/venv/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:213: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


### Vectorization of Env

In [125]:
from stable_baselines3.common.vec_env import VecCheckNan

train_env = DummyVecEnv([lambda: train_env])
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=False)
train_env = VecCheckNan(train_env, raise_exception=True)
# train_env = VecFrameStack(train_env, 15, channels_order='first')

In [126]:
train_env.observation_space.shape, train_env.action_space.shape

((63, 14), (63,))

## stablebaselines Models

In [127]:
from stable_baselines3.ppo import MlpPolicy
from sb3_contrib.ppo_recurrent import MlpLstmPolicy
from sb3_contrib import RecurrentPPO

In [128]:
model = SAC('MlpPolicy', train_env, learning_rate=1e-3,
            verbose=1, tensorboard_log='logs/',
            device='cuda', batch_size=128
            )
model.policy

Using cuda device


/home/hivaze/Documents/CODE-W/PyCharm Projects/portfolio_optimization/venv/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 7.32GB > 0.99GB
  warnings.warn(


SACPolicy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (latent_pi): Sequential(
      (0): Linear(in_features=882, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu): Linear(in_features=256, out_features=63, bias=True)
    (log_std): Linear(in_features=256, out_features=63, bias=True)
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=945, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
    (qf1): Sequential(
      (0): Linear(in_features=945, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=2

In [129]:
mean_reward, std_reward = evaluate_policy(model, train_env, n_eval_episodes=1, deterministic=False, warn=False, return_episode_rewards=True)
print(mean_reward, std_reward)

{'min_reward': -0.13507603275974894, 'max_reward': 0.10769509643262032, 'mean_reward': 0.0012846439338780953, 'portfolio_value': 1.618651356686399, 'weights_q0.25': 0.008902000079256317, 'weights_q0.5': 0.015858724883870457, 'weights_q0.75': 0.022327666168714004, 'weights_max': 0.028236541936997102, 'weights_std': 0.007590259833949908}
[1.6186513373259004] [1260]


### Learn loop

In [130]:
# eval_callback = EvalCallback(eval_env=test_env, best_model_save_path='models/best.stbl',
#                              n_eval_episodes=3, eval_freq=500, render=False, warn=False)
# eval_callback

In [131]:
class MeanRewardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(MeanRewardCallback, self).__init__(verbose)
        self.render = {}

    def _on_step(self) -> bool:

        if self.training_env.get_attr('rewards_memory')[0]:  # else - end of episode
            # reward = self.training_env.get_attr('rewards_memory')[0][-1]
            self.render = self.training_env.render()
        else:
            self.logger.record("train/mean_reward", self.render['mean_reward'])
            self.logger.record("train/min_reward", self.render['min_reward'])
            self.logger.record("train/max_reward", self.render['max_reward'])
            self.logger.record("train/portfolio_value", self.render['portfolio_value'])

        return True

In [133]:
model_name = 'SAC-mr-it-256-256-64-dw'
model.learn(total_timesteps=120_000, progress_bar=False, log_interval=1,
            reset_num_timesteps=False, tb_log_name=model_name,
            callback=MeanRewardCallback())

Logging to logs/SAC-mr-it-256-256-64-dw_0
{'min_reward': -0.13762619245864277, 'max_reward': 0.11277066807282608, 'mean_reward': 0.001214601025250331, 'portfolio_value': 1.530397291815418, 'weights_q0.25': 0.008926459422853462, 'weights_q0.5': 0.014768917028864586, 'weights_q0.75': 0.022431659233683038, 'weights_max': 0.03455021941076303, 'weights_std': 0.008203804591619736}
---------------------------------
| time/              |          |
|    episodes        | 1        |
|    fps             | 123      |
|    time_elapsed    | 5        |
|    total_timesteps | 1260     |
| train/             |          |
|    actor_loss      | -101     |
|    critic_loss     | 1.27     |
|    ent_coef        | 0.316    |
|    ent_coef_loss   | -120     |
|    learning_rate   | 0.001    |
|    max_reward      | 0.113    |
|    mean_reward     | 0.00121  |
|    min_reward      | -0.138   |
|    n_updates       | 1158     |
|    portfolio_value | 1.52     |
---------------------------------
{'min_rewa

KeyboardInterrupt: 

#### Saving model and running mean from env

In [116]:
model.save('models/' + model_name)
with open('models/' + model_name + '_obs_rms.pickle', 'wb') as handle:
    pickle.dump(train_env.obs_rms, handle)

## Evaluation

In [117]:
mean_reward, std_reward = evaluate_policy(model, train_env, n_eval_episodes=5, deterministic=False, warn=False, return_episode_rewards=False)
print(mean_reward, std_reward)

{'min_reward': -0.12093659570347387, 'max_reward': 0.13205487355601875, 'mean_reward': 0.005119468725896201, 'portfolio_value': 6.4505305946292, 'weights_q0.25': 0.0046672422405370905, 'weights_q0.5': 0.004667242240612539, 'weights_q0.75': 0.035565801502095346, 'weights_max': 0.03556582807247249, 'weights_std': 0.014756218279444051}
{'min_reward': -0.12093659570358725, 'max_reward': 0.13205487355601658, 'mean_reward': 0.005118200158448007, 'portfolio_value': 6.448932199644485, 'weights_q0.25': 0.004667275978058943, 'weights_q0.5': 0.004667275978134356, 'weights_q0.75': 0.03556555513505742, 'weights_max': 0.03556558169303568, 'weights_std': 0.01475611168274437}
{'min_reward': -0.12093659580237294, 'max_reward': 0.13205487355601433, 'mean_reward': 0.005117685400974314, 'portfolio_value': 6.448283605227631, 'weights_q0.25': 0.004667316638730944, 'weights_q0.5': 0.004667316638806316, 'weights_q0.75': 0.035565258235646546, 'weights_max': 0.03556528477868274, 'weights_std': 0.014755983179587

In [118]:
train_env.venv.venv.envs[0].__dict__.keys()

dict_keys(['data', 'stock_dim', 'close_prices_pct', 'tickers', 'n_assets', 'objective', 'risk_measure', 'rewards_memory', 'actions_memory', 'portfolio_value', 'observations_range', 'current_step', 'weights', 'reward_range', 'action_space', 'observation_space'])

In [119]:
train_env.training = False
train_env.venv.venv.envs[0].observations_range = (0, 1761) # change to use the test period

In [120]:
import copy

obs = train_env.reset()
weights_states = [train_env.venv.venv.envs[0].weights]
portfolio_values = [0]
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = train_env.step(action)
    if done:
        break
    else:
        weights_states.append(copy.deepcopy(train_env.venv.venv.envs[0].weights))
        portfolio_values.append(train_env.venv.venv.envs[0].portfolio_value)
        print(train_env.render())
len(portfolio_values), len(weights_states)

{'min_reward': -0.006397026892399557, 'max_reward': -0.006397026892399557, 'mean_reward': -0.006397026892399557, 'portfolio_value': -0.006397026892399557, 'weights_q0.25': 0.004027668014186816, 'weights_q0.5': 0.004027668014186816, 'weights_q0.75': 0.029760664904694995, 'weights_max': 0.029760664904694995, 'weights_std': 0.012825912504156965}
{'min_reward': -0.017693875109122646, 'max_reward': -0.006397026892399557, 'mean_reward': -0.012045451000761102, 'portfolio_value': -0.024090902001522203, 'weights_q0.25': 0.003939049938678893, 'weights_q0.5': 0.003939049938678893, 'weights_q0.75': 0.02986456196202846, 'weights_max': 0.02986456196202846, 'weights_std': 0.012921866438747234}
{'min_reward': -0.028942241892391973, 'max_reward': -0.006397026892399557, 'mean_reward': -0.017677714631304724, 'portfolio_value': -0.05303314389391418, 'weights_q0.25': 0.00589781622290701, 'weights_q0.5': 0.00605271942452522, 'weights_q0.75': 0.02890391595058908, 'weights_max': 0.044723883378904095, 'weights

(1761, 1761)

In [121]:
portfolio_state = pd.DataFrame(index=train_env.venv.venv.envs[0].close_prices_pct.index, data=portfolio_values)
portfolio_state.to_csv('portfolios/' + model_name + '_portfolio.csv')

In [122]:
weights_df = pd.DataFrame(index=train_env.venv.venv.envs[0].close_prices_pct.columns, data=weights_states[-1])
weights_df.to_csv('portfolios/' + model_name + '_weights.csv')

## Tests

In [ ]:
with open('data/sp500_data_5y.pickle', 'rb') as f:
    data = pickle.load(f)
    close_prices = pd.DataFrame({k: v['Close'] for k, v in data.items()})

In [ ]:
data['AAPL'].diff()

In [ ]:
rand_w = softmax_normalization(np.random.randn(489))
rand_w

In [ ]:
ones_w = softmax_normalization(np.ones(489))
ones_w

In [ ]:
close_prices.diff()

In [ ]:
(close_prices.pct_change().iloc[48] @ rand_w)

In [ ]:
close_prices.pct_change().dropna().mean() @ rand_w

In [ ]:
(close_prices.pct_change().dropna() @ rand_w).sum()

In [ ]:
(close_prices.pct_change().dropna() @ ones_w).sum()